In [132]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error

In [133]:
df = pd.read_csv('boston.csv')
df.head() # read the first 5 rows in the data

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [134]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(12), int64(2)
memory usage: 55.5 KB


In [135]:
df.isnull().sum() # data have no null values

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
MEDV       0
dtype: int64

In [136]:
df.duplicated().sum() # data have no duplicated values

np.int64(0)

In [137]:
df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [138]:
# remove all outliers
columns_outliers = df.select_dtypes(include=['number']).columns

for col in columns_outliers:
    Q1= df[col].quantile(0.25)
    Q3= df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,214.000000,214.000000,214.000000,214.0,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000
mean,0.228482,7.023364,8.627570,0.0,0.499000,6.214463,60.050000,4.445154,4.518692,310.471963,18.678972,391.507757,10.567804,22.177570
std,0.283614,12.298590,5.237659,0.0,0.057924,0.378653,26.419783,1.760778,1.635041,65.158144,1.671545,6.677139,4.116063,4.097183
min,0.006320,0.000000,1.250000,0.0,0.409000,5.390000,2.900000,1.668600,1.000000,188.000000,14.700000,372.080000,2.980000,11.900000
25%,0.069365,0.000000,5.190000,0.0,0.447000,5.936750,40.175000,2.898350,4.000000,273.750000,17.800000,389.392500,7.347500,19.400000
50%,0.131375,0.000000,7.380000,0.0,0.499000,6.152000,62.500000,4.207000,4.000000,304.000000,18.700000,394.225000,9.840000,22.000000
75%,0.253167,12.500000,9.982500,0.0,0.538000,6.455500,82.875000,5.601375,5.000000,342.500000,19.700000,396.900000,13.135000,24.300000
max,2.242360,45.000000,25.650000,0.0,0.624000,7.412000,100.000000,9.222900,8.000000,437.000000,21.200000,396.900000,21.320000,33.000000


In [139]:
cor= df.corr(numeric_only=True)['MEDV'].sort_values(ascending=False)
print(cor)

MEDV       1.000000
RM         0.684333
ZN         0.307642
DIS        0.154968
RAD        0.120954
B         -0.007701
TAX       -0.267205
CRIM      -0.272352
INDUS     -0.346934
NOX       -0.392227
AGE       -0.413385
PTRATIO   -0.430696
LSTAT     -0.661716
CHAS            NaN
Name: MEDV, dtype: float64


In [140]:
# linear regression 

X = df[['RM','ZN','DIS','RAD']]
y = df['MEDV']
X_train,X_temp,y_train,y_temp = train_test_split(X,y,test_size=0.2,random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

model = LinearRegression()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"MSE Linear Regression MSE: {mse:.4f}")

MSE Linear Regression MSE: 4.6769


In [141]:
polyreg = make_pipeline(PolynomialFeatures(degree=4), LinearRegression())
polyreg.fit(X_train, y_train)

y_pred_poly = polyreg.predict(X_test)
mse_poly = mean_squared_error(y_test, y_pred_poly)
print(f'MSE polynomial regression: {mse_poly:.2f}')

MSE polynomial regression: 17.23


In [142]:
# multiple linear regression

X_multi = np.hstack([X, X**2])
X_train, X_test, y_train, y_test = train_test_split(X_multi, y, test_size=0.2, random_state=42)

multi_reg = LinearRegression()
multi_reg.fit(X_train, y_train)
y_pred_multi = multi_reg.predict(X_test)
mse_multi = mean_squared_error(y_test, y_pred_multi)
print(f"MSE Multiple Linear Regression MSE: {mse_multi:.4f}")

MSE Multiple Linear Regression MSE: 6.1711
